<a href="https://colab.research.google.com/github/kanishkad123/Covid19Visualization/blob/master/DeployModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install colabcode
!pip install fastapi

In [39]:
import re
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [33]:
from pydantic import BaseModel
class InterruptionReason(BaseModel):
    interruption: str 
    correction: str
    class Config:
        schema_extra = {
            "example": {
                "interruption": 'RETURED TO THE FIELD RT OVERPRESSURZIED AND THE LT PACK DEFERRED RT PACK HIGH PRESS CAUTION MESSAGE',
                "correction":"SEA MAINT PERFORMED AN OPERATIONAL CHECK OF THE PACK IN VARIOUS SETTINGS, PACK OPS CHECKS GOOD"
            }
        }


In [34]:
stemmer = PorterStemmer()
def identify_tokens1(interruption):
    comment = re.sub(r'\d',' ',interruption)
    words = nltk.word_tokenize(interruption)
    new_words = [stemmer.stem(word) for word in words]
    comment = ' '.join(new_words)
    return comment

In [48]:
from fastapi import FastAPI
import pickle

app = FastAPI()

@app.on_event("startup")
def load_model():
    global model
    model = pickle.load(open("linearsvc_model.pkl", "rb"))

@app.get('/')
def index():
    return {'message': 'This is the homepage of the API '}


@app.post('/predict')
def get_music_category(data: InterruptionReason):
    received = data.dict()
    interruption = received['interruption']
    correction = received['correction']
    #prediction_list = model.predict([identify_tokens1(interruption + ' ' + correction)]).tolist()
    prob = pd.DataFrame(model.predict_proba([identify_tokens1(interruption + ' ' + correction)]), columns=model.classes_)
    pred = prob.sort_values(by=0,axis=1,ascending=False).iloc[:, 0:3]
    predicted = '1. '+ str(pred.columns[0]) + ' : ' + str(pred._get_value(0, pred.columns[0])) + ' 2. ' + str(pred.columns[1]) + ' : ' + str(pred._get_value(0, pred.columns[1])) + ' 3. ' + str(pred.columns[2]) + ' : ' + str(pred._get_value(0, pred.columns[2]))
    
    return {'prediction': predicted}

In [37]:
from colabcode import ColabCode
server = ColabCode(port=10000, code=False)

In [49]:
server.run_app(app=app)

Public URL: NgrokTunnel: "http://1e35df29d6ec.ngrok.io" -> "http://localhost:10000"


INFO:     Started server process [62]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:10000 (Press CTRL+C to quit)


INFO:     2607:fea8:239f:b780:f947:351d:765f:4f59:0 - "GET / HTTP/1.1" 200 OK
INFO:     2607:fea8:239f:b780:f947:351d:765f:4f59:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2607:fea8:239f:b780:f947:351d:765f:4f59:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2607:fea8:239f:b780:f947:351d:765f:4f59:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2607:fea8:239f:b780:f947:351d:765f:4f59:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2607:fea8:239f:b780:f947:351d:765f:4f59:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     99.252.137.35:0 - "GET / HTTP/1.1" 200 OK
INFO:     99.252.137.35:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     99.252.137.35:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     99.252.137.35:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     99.252.137.35:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2605:8d80:6c0:5abb:9a58:c6b:624f:b129:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2605:8d80:6c0:5abb:9a58:c6b:624f:b129:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     99.252.137.3

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [62]
